In [1]:
pip install numpy pandas yfinance scikit-learn tensorflow alpaca-trade-api

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import alpaca_trade_api as tradeapi

print("All imports resolved successfully!")

All imports resolved successfully!


In [3]:
import os

os.environ["ALPACA_API_KEY"] = "PKREM5M6ZOE7JH3J1HN9"
os.environ["ALPACA_SECRET_KEY"] = "l6t6m75KPTx1i1TeVAClR2jjVF7p50cazB8ZYzKX"

In [4]:
import os
import time
import numpy as np
import yfinance as yf
import pandas as pd
from sklearn.linear_model import LinearRegression
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import logging

# ✅ Fetch API keys from environment variables
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=True) if ALPACA_API_KEY and ALPACA_SECRET_KEY else None

# Set up logging
logging.basicConfig(filename='trading_bot.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# ✅ Function to download stock data with retries
def dataset_loader(stock_name, max_retries=3):
    for attempt in range(1, max_retries + 1):
        try:
            dataset = yf.download(stock_name, start="2010-01-01", end="2025-01-01")
            if dataset.empty:
                raise ValueError(f"No data returned for {stock_name}")
            return dataset['Close']
        except Exception as e:
            print(f"❌ Failed to download {stock_name} data (attempt {attempt}/{max_retries}): {e}")
            time.sleep(2 ** attempt)  # Exponential backoff

    print(f"⚠️ All retries failed. Falling back to mock data for {stock_name}.")
    return pd.Series(100 + np.cumsum(np.random.normal(0, 1, 100)))  # Random walk mock data

# ✅ Place order function
def place_order(symbol, qty, side, limit_price):
    if trading_client:
        try:
            order = LimitOrderRequest(
                symbol=symbol,
                qty=qty,
                side=side,
                time_in_force=TimeInForce.DAY,
                limit_price=round(limit_price, 2)
            )
            trading_client.submit_order(order)
            logging.info(f"✅ REAL ORDER PLACED: {side} {qty} shares of {symbol} at ${round(limit_price, 2)}")
        except Exception as e:
            logging.error(f"❌ Error placing {side} order for {symbol}: {str(e)}")
            if hasattr(e, 'response') and hasattr(e.response, 'json'):
                logging.error(f"API Response: {e.response.json()}")
    else:
        logging.info(f"🔄 Simulated {side} order for {qty} shares of {symbol}.")

# ✅ Machine Learning Trader
class ML_Trader:
    def __init__(self, window_size=10):
        self.window_size = window_size
        self.model = LinearRegression()
        self.inventory = []

    def train(self, data):
        X, y = [], []
        for i in range(len(data) - self.window_size):
            X.append(data.iloc[i:i + self.window_size].values)
            y.append(data.iloc[i + self.window_size])
        X, y = np.array(X), np.array(y)
        if X.shape[0] > 0:
            self.model.fit(X, y)
        else:
            raise ValueError("Not enough data to train the model!")

    def predict(self, recent_data):
        return self.model.predict(np.array(recent_data).reshape(1, -1))[0]

# ✅ Load data and train model
stock_name = 'NVDA'
data = dataset_loader(stock_name)
window_size = 10
trader = ML_Trader(window_size)
trader.train(data)

print(f"Loaded data: {len(data)} points")

# ✅ Ensure correct input shape for R² score calculation
if len(data) > window_size:
    X_test = np.array([data.iloc[i:i + window_size].values for i in range(len(data) - window_size)])
    y_test = np.array(data.iloc[window_size:])
    print(f"Model R² Score: {trader.model.score(X_test, y_test):.4f}")

print("Starting trading loop...")

for t in range(window_size, len(data) - 1):
    recent_data = data.iloc[t - window_size:t].values.flatten()
    predicted_price = trader.predict(recent_data).item()
    actual_price = round(data.iloc[t].item(), 2)

    time.sleep(5)  # Prevents wash trade detection

    # ✅ Buy Condition
    if predicted_price > actual_price and len(trader.inventory) == 0:
        trader.inventory.append(actual_price)
        print(f"AI Trader bought: ${actual_price:.2f}")
        place_order(stock_name, 10, OrderSide.BUY, actual_price)

    # ✅ Sell Condition
    elif predicted_price < actual_price and len(trader.inventory) > 0:
        time.sleep(10)  # Delay before selling
        buy_price = trader.inventory.pop(0)
        profit = round(actual_price - buy_price, 2)
        print(f"AI Trader sold: ${actual_price:.2f}, Profit: ${profit:.2f}")
        place_order(stock_name, 10, OrderSide.SELL, actual_price)

[*********************100%***********************]  1 of 1 completed


❌ Failed to download NVDA data (attempt 1/3): No data returned for NVDA


[*********************100%***********************]  1 of 1 completed


❌ Failed to download NVDA data (attempt 2/3): No data returned for NVDA


[*********************100%***********************]  1 of 1 completed


❌ Failed to download NVDA data (attempt 3/3): No data returned for NVDA
⚠️ All retries failed. Falling back to mock data for NVDA.
Loaded data: 100 points
Model R² Score: 0.9737
Starting trading loop...
AI Trader bought: $98.53
AI Trader sold: $99.15, Profit: $0.62
AI Trader bought: $98.31
AI Trader sold: $97.80, Profit: $-0.51
AI Trader bought: $97.02
AI Trader sold: $97.43, Profit: $0.41
AI Trader bought: $97.78
AI Trader sold: $97.28, Profit: $-0.50
AI Trader bought: $97.17
AI Trader sold: $97.64, Profit: $0.47
AI Trader bought: $97.05
AI Trader sold: $97.79, Profit: $0.74
AI Trader bought: $97.91
AI Trader sold: $98.69, Profit: $0.78
AI Trader bought: $97.87
AI Trader sold: $97.84, Profit: $-0.03
AI Trader bought: $98.94
AI Trader sold: $100.01, Profit: $1.07
AI Trader bought: $99.03
AI Trader sold: $100.64, Profit: $1.61
AI Trader bought: $100.71
AI Trader sold: $94.62, Profit: $-6.09
AI Trader bought: $93.81
AI Trader sold: $88.96, Profit: $-4.85
AI Trader bought: $89.85
AI Trade